<a href="https://colab.research.google.com/github/hanadoolsae/ewhaguidebook/blob/main/data/web_scraping/webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:10 https://ppa.launchpadcon

In [ ]:
import openpyxl
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup


# Chrome driver 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

# 엑셀 파일 로드
workbook = openpyxl.load_workbook("/content/data1.xlsx")
sheet = workbook.active

# ISBN이 저장된 컬럼이 b이고, 개요를 저장할 컬럼이 k라고 가정
for row in range(1, sheet.max_row + 1):
    isbn = sheet[f"B{row}"].value
    page = None


    try:
        # 웹사이트 접속
        driver.get(f"https://product.kyobobook.co.kr/detail/{isbn}")
        time.sleep(2)


        # 책의 설명을 담고 있는 div 태그의 내용 가져오기
        description_element = driver.find_element(By.CSS_SELECTOR, ".info_text:not(.fw_bold)")
        description = description_element.text

        # 이미지 태그 검색
        image_tag = driver.find_element(By.CSS_SELECTOR, "div.blur_img_box img")

        # 이미지 URL 추출
        image_url = image_tag.get_attribute("src")

        # 쪽수를 담고 있는 태그의 내용 가져오기
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')

        # '쪽수' 문자열이 포함된 th 태그를 찾습니다.
        th_tag = soup.find('th', string=lambda s: '쪽수' in s.strip())

        # 찾은 태그의 다음 sibling(동일한 레벨에 있는 다음 태그)을 가져옵니다.
        if th_tag:
            td_tag = th_tag.find_next_sibling('td')
            if td_tag:
                page_info = td_tag.text.strip()
                # 숫자만 추출
                page = ''.join(filter(str.isdigit, page_info))
            else:
                raise ValueError("TD tag for page info not found")
        else:
            raise ValueError("TH tag for '쪽수' not found")


        # 쪽수 출력
        print(f"ISBN {isbn}: Page Found: {page}")
        # 개요 출력
        print(f"ISBN {isbn}: introduce Found: {description}")
        # 이미지 URL 및 태그들 출력
        print(f"ISBN {isbn}: Images Found: {image_url}")


    except Exception as e:
        print(f"Error for ISBN {isbn}: {str(e)}")
        description = ""


    # 이미지 URL 저장
    sheet[f"I{row}"].value = image_url

    # 개요 저장
    sheet[f"K{row}"].value = description

    # 쪽수 저장
    sheet[f"L{row}"].value = page

# 웹드라이버 종료
driver.quit()

# 엑셀 파일 저장
workbook.save("book_data_last.xlsx")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
ISBN S000000598196: Page Found: 304
ISBN S000000598196: introduce Found: “내가 이 책을 읽은 최초의 자폐인인 것 같다. 초고층 빌딩에 올라가 ‘이 책을 읽고 우리를 이해해줘!’라고 온 세상에 외치고 싶다.” _미국 아마존 독자 서평
세계적 뇌과학자 아버지 헨리 마크람과 자폐 아들 카이, 서로를 이해하기 위한 그들의 경이롭고 특별한 여정. 아들 앞에서 무력한 아버지이자 무능한 학자일 수밖에 없었던 헨리는 카이를 위해 자폐증 연구에 매달린다. 이 책은 헨리가 상처와 고통, 실패 끝에 기존의 상식을 뒤집고 마침내 새로운 진실에 도달하는 과정을 솔직하고 담담하게 그렸다. 우리는 모두 다른 세계에 살고 있을 뿐이다. 조금 더 조용하거나, 조금 더 격렬한 세계에.
ISBN S000000598196: Images Found: https://contents.kyobobook.co.kr/sih/fit-in/458x0/pdt/9788934992783.jpg
ISBN S000001941872: Page Found: 212
ISBN S000001941872: introduce Found: ● 어른의 행복은 있다, 분명 있다
마스다 미리는 일상에서 반짝임을 발견해내는 작가이다. 마스다 미리의 에세이는 그 반짝임들의 기록이다. 그런 마스다 미리가 이번 에세이 『행복은 이어달리기』에서는 ‘어른의 일상 속에 숨어 있는 작은 행복’을 발견하고 이야기한다.
그 작은 행복에 대한 이야기는 비록 읽는 순간에는 소소하게 느껴지고, 조금씩 쌓이는 적금처럼 별것 아닌 것 같지만, 결국 우리의 삶을 탄탄하고 윤기있게 만든다.
어느 새 어른이 되어 한숨으로 가득찬 일상을 보내는 우리. 오늘 밀린 업무를 처리하느라, 그러면서 자존감을 회복하느라 바쁘고, 불안감에 노후자금을 마련하느라 빠듯하다. 행복한 일은 하나도 생기지 않는 것만 같다.
어른에게 행복이란 뭘까? 도대체 있기는 한 걸까